# Beta Data Anaysis

We have been working to create a number of queries that describe the contents in the existing enriched-data collection. This notebook can be configured to point to any of the 3 environments (Live, Integration, DevQA).

In [3]:
# Related page
# http://confluence.epnet.com/display/ART/Spike+-+Date+field%28s%29+analyses+by+content+type+in+Raw-Meta

# Start of the script
import pandas as pd
import pymongo
from pymongo import MongoClient

env = input("Please Select an Environment D,I or L: ")
if env == "D":
    mng_svr = '10.25.216.142'
    mng_port = 27017
    mng_user = 'readOnlyUser'
    mng_pwd = 'uk53r8gb22zq76qb'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for DevQA")
elif env == "I":
    mng_svr = '10.25.155.11'
    mng_port = 27017
    mng_user = 'readOnlyUser'
    mng_pwd = 'uk53r8gb22zq76qb'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for Integration")
elif env == "L":
    mng_svr = '10.26.152.183'
    mng_port = 27017
    mng_user = 'readOnlyUser'
    mng_pwd = 'uk53r8gb22zq76qb'
    mng_conn_string = mng_svr,mng_port,mng_user,mng_pwd
    print("Configuring for Live")
else:
     print("No usable environment selected")
     exit()


#mng_conn_string = (mng_svr,mng_port,mng_user,mng_pwd)
print (mng_conn_string)


client = MongoClient (mng_svr,mng_port,username = mng_user, password = mng_pwd)
db = client.documents
collection = db['enriched-meta']




print ("Connection complete")





#=====================================================
#Determine the content types in the collection

#Dist = collection.aggregate([{"$group" : {"_id" : {"type" : "content-type"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
#df = pd.DataFrame(list(Dist))



#Dist = collection.aggregate([{"$group" : {"_id" : {"vertical" : "$header.content-verticals"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
#df = pd.DataFrame(list(Dist))
#data = pd.DataFrame(list(collection.find().limit(1000)))

# End of the script

Please Select an Environment D,I or L: D
Configuring for DevQA
('10.25.216.142', 27017, 'readOnlyUser', 'uk53r8gb22zq76qb')
Connection complete


In [5]:
print("Initial analysis starting")
allRecords = collection.find({}).count()
print("Number of documents: ", allRecords)
mfsRecords = collection.find({"header.content-type":"sch"}).count()
print("Number of MFS documents: ", mfsRecords)
imgRecords = collection.find({"header.content-type":"imh"}).count()
print("Number of Image documents: ", imgRecords)
ersRecords = collection.find({"header.content-type":"ers"}).count()
print("Number of ERS documents: ", ersRecords)
print("Initial analysis complete")

Initial analysis starting
Number of documents:  1148717


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  
C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


Number of MFS documents:  802488


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


Number of Image documents:  277112


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  


Number of ERS documents:  69117
Initial analysis complete


In [3]:
#Determine the content types in the collection
print ("Content Type Analysis starting")
Dist = collection.aggregate([{"$group" : {"_id" : {"type" : "$header.content-type"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
df = pd.DataFrame(list(Dist))
df["Perc"] = df['total'] / allRecords * 100

print ("Content Type Analysis complete")

Content Type Analysis starting
Content Type Analysis complete


In [4]:
df

,_id,total,Perc
0,{'type': ['sch']},802488,69.859565
1,{'type': ['imh']},277111,24.123543
2,{'type': ['ers']},69117,6.016892


Date Analysis; We will look at the prevalance of string dates in MFS fields.


In [10]:
print("Date analysis starting")
mfsRecords_w_date = collection.find({
    '$and' : [{ "header.content-type" :  "sch"},
            {"date.string-date" :  {"$exists" : True}}]
}).count()

Dist = collection.aggregate([{"$match" : { "header.content-type":"sch"}}, {"$group" : { "_id" : "$date.string-date", "total" : {"$sum" : 1} }}, {"$sort": {"total":-1}}])
df = pd.DataFrame(list(Dist))
print("MFSDocuments_W_Date: ", mfsRecords_w_date )
print("Percentage of MFS documents with a String Date: ", mfsRecords_w_date/mfsRecords * 100)
print("Date analysis complete")



Date analysis starting


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  after removing the cwd from sys.path.


MFSDocuments_W_Date:  802488
Percentage of MFS documents with a String Date:  100.0
Date analysis complete


In [11]:
df

,_id,total
0,"[Oct2017, 1506816000000000]",33506
1,"[2005, 1104537600000000]",4355
2,"[8/1/2017, 1501545600000000]",2294
3,"[2003, 1041379200000000]",1934
4,"[Oct2005, 1128124800000000]",1650
5,"[3/1/2006, 1141171200000000]",1620
6,"[Sep2005, 1125532800000000]",1590
7,"[Feb2005, 1107216000000000]",1583
8,"[Apr2005, 1112313600000000]",1577
9,"[Mar2005, 1109635200000000]",1554


In [6]:
#Structured Date field analysis look for Ranges in the Structured date fields (Year, Month, Day)
print("Structured Date analysis starting")
mfsRecords_w_year = collection.find({
    '$and' : [{ "header.content-type" :  "sch"},
    {"date.year" :  {"$exists" : True}}]
}).count()
print("Percentage of MFS documents with a Year: ", mfsRecords_w_year/mfsRecords * 100)

imgRecords_w_year = collection.find({
    '$and' : [{ "header.content-type" :  "imh"},
            {"date.year" :  {"$exists" : True}}]
}).count()
print("Percentage of Image documents with a Year: ", imgRecords_w_year/imgRecords * 100)

ersRecords_w_year = collection.find({
    '$and' : [{ "header.content-type" :  "ers"},
            {"date.year" :  {"$exists" : True}}]
}).count()
print("Percentage of MFS documents with a Year: ", ersRecords_w_year/ersRecords * 100)



Structured Date analysis starting


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


Percentage of MFS documents with a Year:  100.0


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


Percentage of Image documents with a Year:  100.0


C:\Users\smcadoo\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


Percentage of MFS documents with a Year:  100.0
Structured Date analysis complete


In [13]:
print("Year Min / Max  analysis starting")
#year_max = collection.aggregate([
#    {"$match": {"date.date-type": "pub-date" }},
#    {"$group": {"_id:" : "year" , "ym" : {"$max" : "$date.year"}}}])
year_max = collection.aggregate({"$group" : {"_id": "null", "max": { "$max": "$date.day"}}})
print("Highest Year in the dataset: ",year_max)

print("Year Min / Max  analysis complete")

Year Min / Max  analysis starting


TypeError: pipeline must be a list

In [9]:
#Content verticals distribution
print ("Content Vertical Analysis Starting")
Dist = collection.aggregate([{"$group" : {"_id" : {"vertical" : "$header.content-verticals"}, "total" : {"$sum" : 1} }},{"$sort": {"total": -1}}])
df = pd.DataFrame(list(Dist))
df["Perc"] = df['total'] / allRecords * 100
print ("Content Vertical Analysis complete")

Content Vertical Analysis Starting
Content Vertical Analysis complete


In [10]:
df

,_id,total,Perc
0,{'vertical': [['Article']]},484359,42.172411
1,{'vertical': [['Image']]},277110,24.127552
2,"{'vertical': [['Article', 'Scholarly Article']]}",125893,10.961315
3,{'vertical': [['Reference Entry']]},69087,6.015301
4,"{'vertical': [['Reference Entry', 'Article']]}",35183,3.063331
5,{'vertical': [['Book Review']]},21993,1.914898
6,"{'vertical': [['Book Review', 'Review']]}",21514,1.873192
7,"{'vertical': [['Letter', 'Article']]}",17716,1.542506
8,"{'vertical': [['Product Review', 'Review']]}",14378,1.251871
9,{'vertical': [['Letter']]},7057,0.614442


In [5]:
print ("AN Analysis Starting")
Dist = collection.find({"header.content-type" : "sch"},{"article-id-group.CreativeWork-id" : 1}).limit(1000)
df = pd.DataFrame(list(Dist))
print ("AN Analysis complete")

AN Analysis Starting
AN Analysis complete


In [6]:
df

,_id,article-id-group
0,5b8eb3e97f37acef09439127,"[{'CreativeWork-id': '31465077'}, {}]"
1,5b8eb3e97f37acef09439128,"[{'CreativeWork-id': '4115884'}, {}]"
2,5b8eb3e97f37acef09439129,"[{'CreativeWork-id': '10069756'}, {}]"
3,5b8eb3e97f37acef0943912a,"[{'CreativeWork-id': '102921758'}, {'CreativeW..."
4,5b8eb3e97f37acef0943912b,"[{'CreativeWork-id': '58843078'}, {}]"
5,5b8eb3e97f37acef0943912c,"[{'CreativeWork-id': '74118961'}, {}]"
6,5b8eb3e97f37acef0943912d,"[{'CreativeWork-id': '125653400'}, {}]"
7,5b8eb3e97f37acef0943912e,"[{'CreativeWork-id': '24394038'}, {}]"
8,5b8eb3e97f37acef09439131,"[{'CreativeWork-id': '97719850'}, {}]"
9,5b8eb3e97f37acef09439132,"[{'CreativeWork-id': '34782007'}, {}]"


In [ ]:
import cx_Oracle
con = cx_Oracle.connect('aws_reader/mfaws2@mfaws2')
print (con.version)
con.close()

In [3]:
!pip install cx_Oracle --upgrade

  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
!python -m pip install --upgrade pip

  Found existing installation: pip 9.0.1
    Uninstalling pip-9.0.1:
      Successfully uninstalled pip-9.0.1


Cache entry deserialization failed, entry ignored


In [14]:
cx_oracle.version

NameError: name 'cx_oracle' is not defined

In [ ]:
!pip uninstall cx_Oracle